# Payload Logging 

An example of payload logging of Seldon Deployment requests and responses.

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Install Seldon Core as described in [docs](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html)

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: 
 
 ```bash 
 kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080```
 
 * Istio: 
 
 ```bash 
 kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80```
 

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Deploy a Request Logger

This will echo CloudEvents it receives.


In [ ]:
!pygmentize message-dumper.yaml

In [ ]:
!kubectl apply -f message-dumper.yaml -n seldon

## Create a Model with Logging

In [ ]:
!pygmentize model_logger.yaml

In [ ]:
!kubectl apply -f model_logger.yaml -n seldon

## Send a Prediction Request

In [ ]:
!curl -v -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/model-logs/api/v1.0/predictions \
   -H "Content-Type: application/json";

## Check Logger

In [ ]:
!kubectl logs $(kubectl get pods -l run=logger -n seldon -o jsonpath='{.items[0].metadata.name}') 

## Clean Up

In [ ]:
!kubectl delete -f model_logger.yaml -n seldon

In [ ]:
!kubectl delete -f message-dumper.yaml -n seldon